# Tutorial 01 — Build a Basic SOEN Model

This notebook walks through building a simple SOEN model programmatically, feeding it a small input sequence, and plotting state trajectories.

---

## 🔊 NOISE CONFIGURATION: ENABLED (Default)

> **This tutorial runs with NOISE INJECTION (documented defaults).**
>
> | Parameter | Default | Description |
> |-----------|---------|-------------|
> | `phi` | **0.01** | Noise on input flux |
> | `s` | **0.005** | Noise on state |
> | `relative` | **False** | Absolute scaling |
>
> **To toggle noise on/off:** Use the `NOISE_ENABLED` variable in the next code cell.

---

## 0) Setup

In [ ]:
# Setup: Ensure soen_toolkit is importable
import sys
from pathlib import Path

# Add src directory to path if running from notebook location
notebook_dir = Path.cwd()
src_dir = notebook_dir.parent.parent.parent  # tutorial_notebooks -> soen_toolkit -> src -> repo_root/src
if (src_dir / "soen_toolkit").exists():
    sys.path.insert(0, str(src_dir))
else:
    # Try repo root's src directory
    for parent in [notebook_dir] + list(notebook_dir.parents):
        candidate = parent / "src"
        if (candidate / "soen_toolkit").exists():
            sys.path.insert(0, str(candidate))
            break

import matplotlib.pyplot as plt
import torch

from soen_toolkit.core import (
    ConnectionConfig,
    LayerConfig,
    SimulationConfig,
    SOENModelCore,
)
from soen_toolkit.physics.constants import dt_nanoseconds_per_step

In [ ]:
# ==============================================================================
# NOISE CONFIGURATION TOGGLE
# ==============================================================================
# Set NOISE_ENABLED = False to run with ideal conditions (no noise)
# Set NOISE_ENABLED = True for noise injection (default)

NOISE_ENABLED = True  # Toggle this to enable/disable noise

# Default noise parameters (documented defaults)
NOISE_DEFAULTS = {
    "phi": 0.01,           # Noise on input flux
    "s": 0.005,            # Noise on state
    "g": 0.0,              # Source function noise
    "bias_current": 0.0,   # Bias current noise
    "j": 0.0,              # Connection weight noise
    "relative": False,     # Absolute scaling
}

def set_model_noise(model, enabled=True, noise_values=None):
    """
    Toggle noise injection on/off for a SOEN model.
    
    Args:
        model: SOENModelCore instance
        enabled: If True, apply noise; if False, set all noise to 0
        noise_values: Dict of noise parameters (uses NOISE_DEFAULTS if None)
    
    Returns:
        model: The modified model (for chaining)
    """
    from soen_toolkit.core.configs import NoiseConfig
    
    if noise_values is None:
        noise_values = NOISE_DEFAULTS
    
    # Update layer noise configurations
    for cfg in model.layers_config:
        if enabled:
            cfg.noise = NoiseConfig(
                phi=noise_values.get("phi", 0.01),
                s=noise_values.get("s", 0.005),
                g=noise_values.get("g", 0.0),
                bias_current=noise_values.get("bias_current", 0.0),
                j=noise_values.get("j", 0.0),
                relative=noise_values.get("relative", False),
                extras=getattr(cfg.noise, "extras", {}),
            )
        else:
            cfg.noise = NoiseConfig(
                phi=0.0, s=0.0, g=0.0, bias_current=0.0, j=0.0,
                relative=False,
                extras=getattr(cfg.noise, "extras", {}),
            )
    
    # Update connection noise configurations
    for conn_cfg in model.connections_config:
        if enabled:
            conn_cfg.noise = NoiseConfig(
                phi=0.0, g=0.0, s=0.0, bias_current=0.0,
                j=noise_values.get("j", 0.0),
                relative=noise_values.get("relative", False),
                extras={},
            )
        else:
            conn_cfg.noise = NoiseConfig(
                phi=0.0, g=0.0, s=0.0, bias_current=0.0, j=0.0,
                relative=False, extras={},
            )
    
    status = "ENABLED" if enabled else "DISABLED"
    print(f"✓ Noise injection {status}")
    if enabled:
        print(f"  phi={noise_values['phi']}, s={noise_values['s']}, "
              f"relative={noise_values['relative']}")
    
    return model

print(f"Noise injection: {'ENABLED' if NOISE_ENABLED else 'DISABLED'}")
if NOISE_ENABLED:
    print(f"  Default values: phi={NOISE_DEFAULTS['phi']}, s={NOISE_DEFAULTS['s']}, "
          f"relative={NOISE_DEFAULTS['relative']}")

## 1) Define a tiny model

We’ll create a 2-layer network of SingleDendrite nodes/dendrites. We’ll use the Forward Euler solver and the RateArray source function. The first layer is an input-type layer, meaning it is not dynamic, it is not solved or evaluated. It is just a placeholder for the inputs we feed the model. The connection is all-to-all between the 2 layers.

Three things are needed to build a model:
1. An instance of the SimulationConfig defining the core simulation settings.
2. A list of LayerConfigs, defining each layer.
3. A list of connections, defining each connection matrix.


In [ ]:
# Global simulation settings
sim_cfg = SimulationConfig(
    dt=37,  # integration step size in dimensionless units of time
    track_phi=False,  # set True if you want phi (input flux to layers) history recorded
    track_power=True,  # set True to collect power/energy if using SingleDendrite
)

# Layer 0: 8 SingleDendrite neurons, RateArray source, FE solver
layer0 = LayerConfig(
    layer_id=0,  # The layer with ID=0 is treated as the input layer
    layer_type="Linear",
    description="Current sources",
    params={
        "dim": 4,
    },
)

# Layer 1: 4 SingleDendrite neurons
layer1 = LayerConfig(
    layer_id=1,
    layer_type="SingleDendrite",
    description="Readout layer",
    params={
        "dim": 5,
        "solver": "FE",  # if we have no internal connections and use a linear in s source function, then we could also use PS (parallel associative scan)
        "source_func": "RateArray",
        # Optional: tweak defaults if you like (phi_offset, bias_current, gamma_plus, gamma_minus)
        "phi_offset": 0.23,
        "bias_current": 1.7,
        "gamma_plus": {"distribution": "normal", "params": {"mean": 0.001, "std": 0.0}},
        "gamma_minus": {"distribution": "normal", "params": {"mean": 0.001, "std": 0.0}},
    },
)

layers = [layer0, layer1]

# One inter-layer connection: layer 0 → layer 1 (all-to-all)
conn01 = ConnectionConfig(
    from_layer=0,
    to_layer=1,
    connection_type="dense",
    params={
        # weight initializer; normal(0, 0.1) by default; you can switch to e.g. "xavier_uniform"
        # "init": "xavier_uniform",
    },
)

connections = [conn01]

# Build the model
model = SOENModelCore(
    sim_config=sim_cfg,
    layers_config=layers,
    connections_config=connections,
)


model.visualize(show_descriptions=True)

Tips:
- You can visualize the architecture via `model.visualize(...)` if Graphviz is installed (see README for tips).
- The package also supports building from YAML/JSON specs (use `SOENModelCore.build`), and a GUI for composing models.

## 2) Create a toy input sequence

We’ll craft a simple set of waveforms for the first layer (4 channels). External input must be shaped `[batch, seq_len, input_dim]`.




In [ ]:
batch_size = 1
seq_len = 300
input_dim = 4  # must match layer 0 dim

t = torch.linspace(0, 1, steps=seq_len)

# Start with zeros
phi = torch.zeros(batch_size, seq_len, input_dim)

# A couple of simple drivers for demonstration
phi[0, :, 0] = 0.5 * torch.sin(2 * torch.pi * 3.0 * t)  # 3 Hz-ish sine
phi[0, :, 1] = 0.3 * torch.sign(torch.sin(2 * torch.pi * 1.0 * t))  # square-ish +/-0.3
phi[0, :, 2] = 0.2 * torch.sin(2 * torch.pi * 8.0 * t)
# others remain zero

phi.shape

Optional: If you want to pass “state” directly instead of “flux”, set `input_type="state"` in `SimulationConfig` and consider using an `Input` layer as the first layer. In this tutorial we stick to flux input.

## 2b) Optional — Provide initial states in forward

Initial states default to zeros. You can provide per-layer initial states at forward time without changing the model definition:


In [ ]:
# Map layer_id -> initial state vector (length must match that layer's dim)
init = {
    0: torch.full((input_dim,), 0.2),  # layer 0 initial s
    # 1: torch.zeros(4),        # example for layer 1
}

with torch.no_grad():
    final_history, all_histories = model(phi, initial_states=init)

## 3) Run a forward pass (simulate dynamics)

The forward pass returns two items:
- `final_history`: the state trajectory of the last layer (`[batch, seq_len + 1, final_dim]`).
- `all_histories`: list of all layers’ state histories (each `[batch, seq_len + 1, dim]`). The first time index is the initial state at t=0.

In [ ]:
model.eval()  # ensures we’re in inference mode
with torch.no_grad():
    final_history, all_histories = model(phi)

len(all_histories), all_histories[0].shape, all_histories[1].shape

## 4) Plot state trajectories per layer

We’ll overlay neuron trajectories for each layer and use a physical time axis in ns with the mapping 1 dt = 1.28 ps. That means one simulation step takes `model.dt × 1.28 ps`. There are `seq_len + 1` points because the solver includes the initial state at t=0.


In [ ]:
import numpy as np

# Physical time per step: convert using physics constants (omega_c)
dt_ns = dt_nanoseconds_per_step(model.dt)

fig, axes = plt.subplots(1, len(all_histories), figsize=(14, 4), sharex=False)
if len(all_histories) == 1:
    axes = [axes]

for li, s_hist in enumerate(all_histories):
    ax = axes[li]
    s = s_hist[0]  # [seq_len+1, dim]

    time_ns = np.arange(s.shape[0]) * dt_ns

    for d in range(s.shape[-1]):
        ax.plot(time_ns, s[:, d].cpu().numpy(), lw=1.2, alpha=0.9, label=f"n{d}" if d < 6 else None)
    ax.set_title(f"Layer {li} states (dim={s.shape[-1]})")
    ax.set_xlabel("Time (ns)")
    ax.set_ylabel("s")
    ax.grid(True, alpha=0.3)
    if s.shape[-1] <= 6:
        ax.legend(loc="upper right", fontsize=8)

plt.tight_layout()
plt.show()

If you enabled `track_s=True`, the layers also store their own `state_history`, but using the returned `all_histories` is most direct for plotting.

## 5) Visualize the architecture

This requires Graphviz installed (see README troubleshooting).

In [ ]:
# examples
model.visualize(file_format="png", simple_view=True, show_desc=True, theme="dark")
model.visualize(file_format="png", simple_view=False, show_desc=True, theme="dark")
model.visualize(file_format="png", simple_view=False, show_desc=False, theme="modern", show_node_ids=True)
# etc...

# You can also view layers/modules as grids. Each node now has a physical location. This spatial structure is used in the
# power-law connectivity pattern. Although when connecting between layers we assume they are 1D lines and use Manhattan distance,
# and when connecting within a layer we treat it as a grid and use Euclidean distance.
model.visualize_grid_of_grids()

## 6) Track power/energy (SingleDendrite)

For SingleDendrite layers you can track power/energy by setting `track_power=True` in `SimulationConfig`. Rebuild the model (or update via `model.set_tracking(track_power=True)`) and rerun the forward pass. The layer will populate `power_bias`, `power_diss`, `energy_bias`, and `energy_diss` in physical units by default. Shapes: `[batch, seq_len, dim]`.


In [ ]:
import numpy as np

# Ensure power tracking is enabled and recompute
model.set_tracking(track_power=True)
with torch.no_grad():
    _ = model(phi)

# Physical time per step: convert using physics constants (omega_c)
DT_NS = dt_nanoseconds_per_step(model.dt)

for li, layer in enumerate(model.layers):
    pb = getattr(layer, "power_bias", None)
    pd = getattr(layer, "power_diss", None)
    eb = getattr(layer, "energy_bias", None)
    ed = getattr(layer, "energy_diss", None)

    if pb is None or pd is None or eb is None or ed is None:
        continue

    # Shapes: [batch, seq_len, dim]
    # Convert to numpy and sum over dendrites
    total_power = (pb + pd)[0].sum(dim=-1).cpu().numpy()  # Watts
    total_energy = (eb + ed)[0].sum(dim=-1).cpu().numpy()  # Joules

    # Time axis in ns using model.dt mapping
    time_ns = np.arange(len(total_power)) * DT_NS

    # Plot power
    plt.figure(figsize=(7, 3))
    plt.plot(time_ns, total_power * 1e9, lw=1.5, color="black")  # W → nW
    plt.title(f"Layer {li}: Total Power vs Time")
    plt.xlabel("Time (ns)")
    plt.ylabel("Power (nW)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Plot energy (cumulative)
    plt.figure(figsize=(7, 3))
    plt.plot(time_ns, total_energy * 1e15, lw=1.5, color="black")  # J → fJ
    plt.title(f"Layer {li}: Total Energy vs Time")
    plt.xlabel("Time (ns)")
    plt.ylabel("Energy (fJ)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

---